# 04 - Création d'un dataset des données nettoyées de job_events (après fractionnement)

Ce notebook génère 1 csv :

- clean_merge_job_events_dataset.csv

Etapes :

- Suppression des lignes avec les valeurs timestamp_start manquantes

- Suppression des colonnes ayant des valeurs uniques

- Remplacement des données maquantes

- Conversion des types des séries en fonction des types des valeurs

# A. Imports

## a) Librairies

In [1]:
import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## b) Données

In [2]:
# source path to raw metrics dataset
filename = 'raw_merge_job_events_dataset.csv'
path = '../data/jobs'
job_events = os.path.join(path, filename)
# target path to save merge raw metrics dataset
save_csv = '../data/jobs/clean_merge_job_events_dataset.csv'

# B. Dataframe

## a) Création

In [3]:
# création d'un dataframe à partir du csv de données
df = pd.read_csv(job_events, index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4232 entries, 0 to 4231
Data columns (total 42 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   jobId                                                 4232 non-null   int64  
 1   timestamp_start                                       4225 non-null   object 
 2   jsonVersion_x                                         4225 non-null   float64
 3   totalCopies_start                                     4225 non-null   float64
 4   LED_iper                                              4225 non-null   float64
 5   bars_iper                                             4225 non-null   object 
 6   drops_iper                                            4225 non-null   float64
 7   dithering_iper                                        4225 non-null   object 
 8   deadPixelsOffset_iper                                 4225

In [4]:
# réindexation
df.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df.head(5)

,jobId,timestamp_start,jsonVersion_x,totalCopies_start,LED_iper,bars_iper,drops_iper,dithering_iper,deadPixelsOffset_iper,level_user,...,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,timestamp_end,totalCopies_end,consumption_operatorSideTanks_varnishConsumption,path,image,jsonVersion_y
0,1666686314,2022-10-25T08:25:14.7283523Z,2.0,1.0,100.0,"[1, 2]",7.0,False,0.0,Operator,...,16.0,False,1.0,SUCCESS,2022-10-25T08:26:18.5227471Z,1.0,0.046652,D:/IMAGES/Standard/22100077-00 test vernis__VE...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,2.0
1,1666686457,2022-10-25T08:27:37.3878834Z,2.0,4.0,100.0,"[1, 2]",6.0,False,0.0,Operator,...,16.0,False,1.0,SUCCESS,2022-10-25T08:28:49.0183229Z,4.0,0.158389,D:/IMAGES/Standard/22100077-00 test vernis__VE...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,2.0
2,1666686620,2022-10-25T08:30:20.2103584Z,2.0,1.0,100.0,"[1, 2]",6.0,False,0.0,Operator,...,16.0,False,1.0,SUCCESS,2022-10-25T08:31:20.8591697Z,1.0,0.368169,D:/IMAGES/Standard/22100077-00 test vernis__VE...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,2.0
3,1666686725,2022-10-25T08:32:05.6033789Z,2.0,4.0,100.0,"[1, 2]",6.0,False,0.0,Operator,...,16.0,False,1.0,SUCCESS,2022-10-25T08:33:24.6918780Z,4.0,1.471035,D:/IMAGES/Standard/22100077-00 test vernis__VE...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,2.0
4,1666694790,2022-10-25T10:46:30.3941268Z,2.0,4.0,100.0,"[1, 2]",6.0,False,0.0,Operator,...,16.0,False,1.0,SUCCESS,2022-10-25T10:48:54.0399761Z,4.0,1.468951,D:/IMAGES/Standard/22100077-00 test vernis__VE...,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAgGBgcGBQgHBw...,2.0


## b) Nettoyage

### 1. Suppression de lignes

In [5]:
# on cherche les index des lignes sans timestamp de départ
no_start_row = list(df[df['timestamp_start'].isna()].index)
no_start_row

[4225, 4226, 4227, 4228, 4229, 4230, 4231]

In [6]:
# on supprime les lignes sans timestamp de départ
df.drop(no_start_row, axis=0, inplace=True)

### 2. Suppression de colonnes

In [7]:
# liste des colonnes
df.columns

Index(['jobId', 'timestamp_start', 'jsonVersion_x', 'totalCopies_start',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'stampAreas_ifoil',
       'heater1Enabled_ifoil', 'speedTensionIn_ifoil', 'speedTensionOut_ifoil',
       'heater1Temperature_ifoil', 'x_imageLayout_layout',
       'y_imageLayout_layout', 'width_paperFormat_layout',
       'height_paperFormat_layout', 'speed_layout', 'power_irDryers',
       'power_uvDryers', 'topMargin_registration_remoteScannerRegistration',
       'leftMargin_registration_remoteScannerRegistration',
       'redScore_gridMode_remoteScannerRegistration',
       'redScore_cropmarksMode_remoteScannerRegistration',
       'redScore_fullScannerMode_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'enable_spec

In [8]:
# liste des types des colonnes
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [9]:
# suppression des colonnes non pertinentes
col_to_drop = [
    'path', 
    'image']
df.drop(col_to_drop, axis=1, inplace=True)

### 3. Données maquantes

In [10]:
# on liste les colonnes qui contiennent des valeurs nulles
col_with_nan = []
for col in df.columns:
    if df[col].isnull().any():
        col_with_nan.append(col)
        print(f'{col} : {df[col].unique()}') if df[col].nunique() <= 10 else print(f'{col} : {df[col].nunique()} - dtype : {df[col].dtype}')     

jobState : ['SUCCESS' 'CANCELED' 'ERROR' nan]
timestamp_end : 4216 - dtype : object
totalCopies_end : 232 - dtype : float64
consumption_operatorSideTanks_varnishConsumption : 3577 - dtype : float64


In [11]:
# on remplace les 'name_paperFormat' sans nom par UNDEFINED
# df['name_paperFormat_layout'] = df['name_paperFormat_layout'].fillna('UNDEFINED')

In [12]:
# on remplace les 'jobState' non défini par UNDEFINED
df['jobState'] = df['jobState'].fillna('UNDEFINED')

In [13]:
# on converti les valeur de 'timestamp_end' en booléen
#df['timestamp_end'] = df['timestamp_end'].apply(lambda x: False if pd.isna(x) else True)
df['timestamp_end'] = df['timestamp_end'].fillna(df['timestamp_start'])

In [14]:
# on remplace les valeurs de 'totalCopies_end' nulles par 0
df['totalCopies_end'] = df['totalCopies_end'].fillna(float(0))

In [15]:
# on remplace les valeurs de 'varnishConsumption' nulles par 0
df['consumption_operatorSideTanks_varnishConsumption'] = df['consumption_operatorSideTanks_varnishConsumption'].fillna(float(0))

### 4. Conversion de type

In [16]:
# conversion des colonnes de type 'float' dont toutes les valeurs ont des décimales à 0 en type 'int'
for col in list(df.select_dtypes(exclude=['object','bool']).columns):
    if not True in (math.modf(value)[0] != float(0) for value in list(df[col].unique())):
        df[col] = df[col].astype('int64')

In [17]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

timestamp_start : 4225 values
bars_iper : ['[1, 2]' '[1]' '[2]']
dithering_iper : [False True]
level_user : ['Operator' 'Distributor']
operator_user : ['User' 'Distributor']
enabled_ifoil : [False True]
optifoil_ifoil : [True False]
stampAreas_ifoil : 58 values
heater1Enabled_ifoil : [False True]
enable_specialSubstrate_remoteScannerRegistration : [False True]
jobState : ['SUCCESS' 'CANCELED' 'ERROR' 'UNDEFINED']
timestamp_end : 4225 values


In [18]:
# on peut convertir certaines colonnes au format datetime
df['timestamp_start'] = pd.to_datetime(df['timestamp_start'], utc=True)
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'], utc=True)

In [19]:
# on peut convertir le type des colonnes contenant True au format booléen
for col in list(df.select_dtypes(include=['object']).columns):
    values = list(df[col].unique())
    if True in (len(values) == 2  and value == True for value in values):
        df[col] = df[col].astype('bool')

In [20]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in list(df.select_dtypes(include=['object']).columns):
    print(f"{col} : {df[col].unique()}") if df[col].nunique() <= 5  else print(f"{col} : {df[col].nunique()} values")

bars_iper : ['[1, 2]' '[1]' '[2]']
level_user : ['Operator' 'Distributor']
operator_user : ['User' 'Distributor']
stampAreas_ifoil : 58 values
jobState : ['SUCCESS' 'CANCELED' 'ERROR' 'UNDEFINED']


### 5. Output csv

In [21]:
df.columns

Index(['jobId', 'timestamp_start', 'jsonVersion_x', 'totalCopies_start',
       'LED_iper', 'bars_iper', 'drops_iper', 'dithering_iper',
       'deadPixelsOffset_iper', 'level_user', 'operator_user', 'speed_ifoil',
       'enabled_ifoil', 'optifoil_ifoil', 'stampAreas_ifoil',
       'heater1Enabled_ifoil', 'speedTensionIn_ifoil', 'speedTensionOut_ifoil',
       'heater1Temperature_ifoil', 'x_imageLayout_layout',
       'y_imageLayout_layout', 'width_paperFormat_layout',
       'height_paperFormat_layout', 'speed_layout', 'power_irDryers',
       'power_uvDryers', 'topMargin_registration_remoteScannerRegistration',
       'leftMargin_registration_remoteScannerRegistration',
       'redScore_gridMode_remoteScannerRegistration',
       'redScore_cropmarksMode_remoteScannerRegistration',
       'redScore_fullScannerMode_remoteScannerRegistration',
       'blueScore_fullScannerMode_remoteScannerRegistration',
       'greenScore_fullScannerMode_remoteScannerRegistration',
       'enable_spec

In [22]:
df.to_csv(path_or_buf=Path(save_csv))